# INF5007 Neural Networks / Neuroniniai tinklai
**LAB8, Part 2**

In [1]:
import numpy as np
import pandas as pd
from tensorflow import keras
import matplotlib.pyplot as plt

## Encoder-Decoder structure for LSTM

In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/vytkuc/inf4039_2023_autumn/master/lab7_RNN/lab_data.csv')
df

,Date,MedTemp
0,2007-11-01,16.15
1,2007-11-02,20.45
2,2007-11-03,18.55
3,2007-11-04,14.40
4,2007-11-05,11.85
...,...,...
3423,2017-06-21,6.15
3424,2017-06-22,6.75
3425,2017-06-23,5.20
3426,2017-06-24,7.25


In [3]:
import plotly 
import plotly.express as px
import plotly.graph_objects as go

def plot_timeseries(date, temp):
    # Plot daily median temperatures in Canberra
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=date, 
                            y=temp,
                            mode='lines',
                            name='Median Temperature',
                            opacity=0.8,
                            line=dict(color='black', width=1)
                            ))

    # Change chart background color
    fig.update_layout(dict(plot_bgcolor = 'white'))

    # Update axes lines
    fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='lightgrey', 
                    zeroline=True, zerolinewidth=1, zerolinecolor='lightgrey', 
                    showline=True, linewidth=1, linecolor='black',
                    title='Date'
                    )

    fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='lightgrey', 
                    zeroline=True, zerolinewidth=1, zerolinecolor='lightgrey', 
                    showline=True, linewidth=1, linecolor='black',
                    title='Degrees Celsius'
                    )

    # Set figure title
    fig.update_layout(title=dict(text="Median Daily Temperatures in Canberra", 
                                font=dict(color='black')))

    fig.show()

plot_timeseries(df.Date, df.MedTemp)

In [4]:
# Normalization
from sklearn.preprocessing import MinMaxScaler

#data_array = np.array(df.MedTemp.astype('float32'))
X = np.array(df.MedTemp).reshape(-1, 1)
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# Train / Test split (initial only)
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(X_scaled, test_size=0.2, shuffle=False)
print(train_data.shape, test_data.shape)

def get_XY(dat, time_steps):
    # Prepare Y
    Y_ind = np.arange(time_steps, len(dat), time_steps)
    Y = dat[Y_ind]

    # Prepare X
    rows_x = len(Y)
    X = dat[range(time_steps*rows_x)]
    X = np.reshape(X, (rows_x, time_steps, 1))    
    
    return X, Y
 
time_steps = 7
trainX, trainY = get_XY(train_data, time_steps)
testX, testY = get_XY(test_data, time_steps)

(2742, 1) (686, 1)


In [5]:
from keras.models import Sequential
from keras import layers

np.random.seed(1330)

def create_model():
    model = keras.Sequential( 
        [
            layers.LSTM(50, activation='tanh', input_shape=(time_steps, 1)),
            layers.Dense(units = 1, activation = "linear"),
            layers.RepeatVector(1),
            layers.LSTM(50, activation='tanh', return_sequences=True),
            layers.TimeDistributed(layers.Dense(1))
        ]
    )
    return model


model = create_model()
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 50)                10400     
                                                                 
 dense (Dense)               (None, 1)                 51        
                                                                 
 repeat_vector (RepeatVector  (None, 1, 1)             0         
 )                                                               
                                                                 
 lstm_1 (LSTM)               (None, 1, 50)             10400     
                                                                 
 time_distributed (TimeDistr  (None, 1, 1)             51        
 ibuted)                                                         
                                                                 
Total params: 20,902
Trainable params: 20,902
Non-traina

In [6]:
model.compile(loss='mean_squared_error', 
              metrics=['MeanSquaredError', 'MeanAbsoluteError'], 
              optimizer='adam')

model.fit(trainX,
          trainY,
          batch_size=5,
          epochs=20,
          verbose=1,
          shuffle=True,
         )

Epoch 1/20


2022-11-15 09:23:48.936049: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


79/79 [==============================] - 2s 2ms/step - loss: 0.0768 - mean_squared_error: 0.0768 - mean_absolute_error: 0.2068
Epoch 2/20
79/79 [==============================] - 0s 1ms/step - loss: 0.0121 - mean_squared_error: 0.0121 - mean_absolute_error: 0.0895
Epoch 3/20
79/79 [==============================] - 0s 1ms/step - loss: 0.0111 - mean_squared_error: 0.0111 - mean_absolute_error: 0.0858
Epoch 4/20
79/79 [==============================] - 0s 1ms/step - loss: 0.0112 - mean_squared_error: 0.0112 - mean_absolute_error: 0.0856
Epoch 5/20
79/79 [==============================] - 0s 1ms/step - loss: 0.0114 - mean_squared_error: 0.0114 - mean_absolute_error: 0.0869
Epoch 6/20
79/79 [==============================] - 0s 1ms/step - loss: 0.0108 - mean_squared_error: 0.0108 - mean_absolute_error: 0.0832
Epoch 7/20
79/79 [==============================] - 0s 1ms/step - loss: 0.0114 - mean_squared_error: 0.0114 - mean_absolute_error: 0.0850
Epoch 8/20
79/79 [===========================

In [7]:
from sklearn.metrics import mean_squared_error 

pred_train = model.predict(trainX).reshape(-1, 1)
pred_test = model.predict(testX).reshape(-1, 1)

print('---------- Evaluation on Training Data ----------')
print("MSE =", mean_squared_error(trainY, pred_train))
print("")

print('---------- Evaluation on Test Data ----------')
print("MSE =", mean_squared_error(testY, pred_test))
print("")

4/4 [==============================] - 0s 704us/step
---------- Evaluation on Training Data ----------
MSE = 0.007745011227810659

---------- Evaluation on Test Data ----------
MSE = 0.00873108252705014



In [8]:
def plot_predicted(true_data, pred_data):
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=np.array(range(0,len(true_data))),
                            y=scaler.inverse_transform(true_data).flatten(),
                            mode='lines',
                            name='Median Temperature - Actual (Test)',
                            opacity=0.8,
                            line=dict(color='black', width=1)
                            ))
    fig.add_trace(go.Scatter(x=np.array(range(0,len(pred_data))),
                            y=scaler.inverse_transform(pred_data).flatten(),
                            mode='lines',
                            name='Median Temperature - Predicted (Test)',
                            opacity=0.8,
                            line=dict(color='red', width=1)
                            ))

    # Change chart background color
    fig.update_layout(dict(plot_bgcolor = 'white'))

    # Update axes lines
    fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='lightgrey', 
                    zeroline=True, zerolinewidth=1, zerolinecolor='lightgrey', 
                    showline=True, linewidth=1, linecolor='black',
                    title='Observation'
                    )

    fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='lightgrey', 
                    zeroline=True, zerolinewidth=1, zerolinecolor='lightgrey', 
                    showline=True, linewidth=1, linecolor='black',
                    title='Degrees Celsius'
                    )

    # Set figure title
    fig.update_layout(title=dict(text="Median Daily Temperatures in Canberra", 
                                font=dict(color='black')),
                    legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1)
                    )

    fig.show()

plot_predicted(testY, pred_test)